# Import, compile data and preprocess

In [10]:
# importations
import numpy as np
import pandas as pd
from pyedflib import EdfReader
import mne

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

import copy
import os
import importlib

import generisAPI
importlib.reload(generisAPI)

import generisAPI.file_operators as fp
importlib.reload(generisAPI.file_operators)

import generisAPI.data_cleaning as dcln
importlib.reload(generisAPI.data_cleaning)

<module 'generisAPI.data_cleaning' from 'd:\\Emmanuel_petron_Olateju\\MMN_SZ\\notebooks\\generisAPI\\data_cleaning.py'>

In [2]:
# definitions
electrodes=['Fp1','FpZ','Fp2','F7','F8','T7','T8','P7','P8','O1','O2','OZ','FZ','PZ','CZ','F3','F4','P3','P4','C3','C4']
len(electrodes)

21

In [11]:
# compile eeg_data and annotations
subjects_dataset_root_path = '../acquired_dataset/'

data = fp.xtract_id_subject_info_dict(subjects_dataset_root_path)

In [13]:
# assert eeg_data trials match marker trials

trialMarkerLength = dcln.assertTrialsMarkersLength()
trialMarkerLength.fit(data)
print(trialMarkerLength.indices)
clean_data = trialMarkerLength.transform(data,False)

[]


In [14]:
# check for null eeg_data

null_data_checker = dcln.nullIndicesTracer()
null_data_checker.fit(clean_data)
print(null_data_checker.indices)
clean_data = null_data_checker.transform(clean_data,True)

{'16': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '20': {'rest1': [], 'arith': [], 'rest2': [], 'auditory': [0]}, '23': {'rest1': [], 'arith': [0], 'rest2': [0], 'auditory': [0]}, '32': {'rest1': [0, 1, 2], 'arith': [0, 1, 2], 'rest2': [0, 1, 2], 'auditory': [0, 1, 2]}, '4': {'rest1': [], 'arith': [], 'rest2': [2], 'auditory': [2]}, '5': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '6': {'rest1': [], 'arith': [], 'rest2': [], 'auditory': [0]}, '8': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '9': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}}


# Preprocessing

In [ ]:
all_subjects_baseline_correction(all_patients_trials_data)

# Phase4 MMN

## all subjects

#### data

In [ ]:
mmn_epoch=Epoch(100,1.3)
sp4_epochs=mmn_epoch.all_subject_phase_epochs(4,all_patients_trials_data,all_patients_annotation,0.2)

sp4_epochs_filtered = filter_epochs(sp4_epochs,1,30,100)

sp4_epochs_average = mmn_epoch.average_epochs(sp4_epochs_filtered)

#### cartesian plot

In [ ]:
_Fs=100
def plot_by_channels(t0,t1,offset,data,electrodes,fig_size=(20,10)):
    assert isinstance(data,dict)
    if (t0<0) or (t0-offset<0):
        t0 = int(t0*_Fs)
        t1 = int(t1*_Fs)
    else:
        t0 = int((t0-offset)*_Fs)
        t1 = int((t1-offset)*_Fs)
    offset=int(offset*_Fs)
    # no_chs = data[list(data.keys())[0]].shape[0]
    no_chs = len(electrodes)
    if no_chs <= 8:
        n_rows = 2
    elif no_chs <= 16:
        n_rows = 4
    elif no_chs <= 24:
        n_rows = 6
    n_cols = int(no_chs/n_rows)
    fig, ax = plt.subplots(n_rows,n_cols,figsize=fig_size)

    for r in range(n_rows):
        for c in range(n_cols):
            lgnd=list()
            for k in data.keys():
                # if k != ' ':
                lgnd.append(k)
                ax[r,c].plot(data[k][(r*n_cols)+c,t0:t1],'--')
                plt.legend(lgnd)
                ax[r,c].set_xticks(list(range(0,data[k][(r*n_cols)+c,t0:t1].shape[0],10)))
                # ax[r,c].set_xticklabels(list(range(-1*offset,t1-offset+10,10)))
                ax[r,c].set_xticklabels(list(range(t0,t1,10)))
                ax[r,c].set_title(electrodes[(r*n_cols)+c])
                ax[r,c].set_xlabel('ms')
                ax[r,c].set_ylabel('uV')

In [ ]:
mmn_epoch.plot_by_channels(0,0.5,0.2,sp4_epochs_average[0],electrodes,(22,12))
# plot_by_channels(0,0.5,0.2,sp4_epochs_average[0],electrodes,(22,12))

#### montage plot

In [ ]:
print(sp4_epochs_average[0].keys(), ', ', len(sp4_epochs_average[0].keys()))

In [ ]:
_Fs=100

def extract_eeg_time_window(eeg_samples,t0,t1,offset=0,_Fs=100):
    if (t0<0) or (t0-offset<0):
        t0 = int(t0*_Fs)
        t1 = int(t1*_Fs)
    else:
        t0 = int((t0-offset)*_Fs)
        t1 = int((t1-offset)*_Fs)
    return eeg_samples[:,t0:t1]

def time_montage_plot(eeg_samples,electrodes,t0,t1,offset,_Fs,fig_size=(6,6)):
    assert isinstance(eeg_samples,dict)
    for k in eeg_samples:
        try:
            assert eeg_samples[k][0:21,:].shape[0] == len(electrodes)
            print(extract_eeg_time_window(eeg_samples[k][0:21,:],t0,t1,offset,_Fs).shape)
        except AssertionError:
            print(eeg_samples[k].shape,', ', len(electrodes))
    n_rows = len(eeg_samples)
    n_cols = int((t1-t0)*_Fs)

    print(n_rows,n_cols)

    fig,ax = plt.subplots(n_rows,n_cols,figsize=fig_size)

    for r,k in enumerate(eeg_samples.keys()):
        for t in range(n_cols):
            # print(r,t)
            montage_plot(
                extract_eeg_time_window(eeg_samples[k][0:21,:],t0,t1,offset,_Fs)[:,t],
                electrodes,ax[r,t])
    plt.show()

In [ ]:
time_montage_plot(sp4_epochs_average[0],electrodes,0,0.03,0.2,100)

## subject specific

### simiat

In [ ]:
s_mmn_epoch=Epoch(100,1.3)

s2p4_epochs=s_mmn_epoch.subject_phase_epochs(0,4,all_patients_trials_data,all_patients_annotation,0.3)

s2p4_epochs_filtered = filter_epochs(s2p4_epochs,1,30,100)

s2p4_epochs_average = s_mmn_epoch.average_epochs(s2p4_epochs_filtered)

mmn_epoch.plot_by_channels(0,50,0.2,s2p4_epochs_average[0],electrodes,(22,12))

### feyisayo

In [ ]:
s_mmn_epoch=Epoch(100,1.3)

s3p4_epochs=s_mmn_epoch.subject_phase_epochs(1,4,all_patients_trials_data,all_patients_annotation,0.3)

s3p4_epochs_filtered = filter_epochs(s3p4_epochs,1,30,100)

s3p4_epochs_average = s_mmn_epoch.average_epochs(s3p4_epochs_filtered)

mmn_epoch.plot_by_channels(0,50,0.2,s3p4_epochs_average[0],electrodes,(22,12))

### abimbola

In [ ]:
s_mmn_epoch=Epoch(100,1.3)

s4p4_epochs=s_mmn_epoch.subject_phase_epochs(2,4,all_patients_trials_data,all_patients_annotation,0.3)

s4p4_epochs_filtered = filter_epochs(s4p4_epochs,1,30,100)

s4p4_epochs_average = s_mmn_epoch.average_epochs(s4p4_epochs_filtered)

mmn_epoch.plot_by_channels(0,50,0.2,s4p4_epochs_average[0],electrodes,(22,12))

# plots

In [ ]:
x=all_patients_trials_data[1][1][4][0:21,0]

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(4,4))
montage_plot(x,electrodes,ax)